In [1]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from AP_class import APDFT_perturbator as AP
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np
import ipyvolume as ipv
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [2]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP
from ABSE import absec
from Morse import Morse_interpolator

In [3]:
from berny import Berny, geomlib, coords,Geometry,Math
import berny

In [4]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [75]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic


ebde=100/627.5 # empirical bond dissociation energy
ch4coords,nh3optg,oh2optg,hfoptg=np.load("optgeoms.npy")
def mpa_pb_with_energy(coords,atoml,g,h,e,gic=False,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    if not gic:
        g_ic=np.dot(B_inv.T, (g).reshape(-1))
    else:
        g_ic=g
    print(g_ic)
    h_ic=build_h_ic(s,g_ic,h,B_inv)
    bnr=0
    dde_mb=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                MorseInt=Morse_interpolator(g_ic[i],h_ic[i,i],q0[i],ebde,e)
                ddq_mb[i]+=MorseInt.re+g_ic[i]/h_ic[i,i]-q0[i]
                dde_mb+=MorseInt.e_min-e+1/2*h_ic[i,i]*(MorseInt.re-q0[i])**2
    dq_NR=-Math.pinv(h_ic)@g_ic
    ddq_mb_old=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    ddq_mb[bnr:]=-Math.pinv(h_ic[bnr:,bnr:]) @(h_ic[bnr:,:]@(ddq_mb))
    print(ddq_mb,ddq_mb_old)
    dq_tot=dq_NR+ddq_mb
    en_MB=(e-1/2*np.einsum("i,ij,j",dq_tot,h_ic,dq_tot)+dde_mb)
    print (s.coords._coords)
    #return q0,q0+dq_tot,en_MB
    return q0,dq_NR,q0+dq_NR,q0+dq_tot,en_MB

In [6]:
HF=gto.M(atom="F 0 0 0;H 0 0 1.695741",unit="bohr",
            basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])})
mfhf = scf.RHF(HF)
mfhf.scf()

converged SCF energy = -100.065540168732


-100.06554016873244

In [7]:
dm=mfhf.make_rdm1()

In [8]:
def Vne(coords,mol=HF,dm=dm):
    mol.set_rinv_orig_(coords)
    dV=mol.intor('int1e_rinv')
    return -np.einsum('ij,ij',dV,dm)

In [9]:
dL=[-1,0,0,0] #-1 charge central atom
def Vnn(coords,mol=HF):
    Vnn=0
    for i in range (mol.natm):
        #Vnn+= (mol.atom_charge(i))/np.linalg.norm(coords-mol.atom_coord(i))        
        Vnn+= (mol.atom_charge(i)+dL[i])/np.linalg.norm(coords-mol.atom_coord(i))
    return Vnn

In [10]:
def Vtot(coords,mol=HF,dm=dm):
    return Vnn(coords,mol)+Vne(coords,mol,dm)

In [11]:
def euler2coords(teta,phi,r):
    return np.asarray([np.cos(phi)*np.sin(teta),np.sin(teta)*np.sin(phi),np.cos(teta)])*r
def Vtot_fl(angles,r,mol=HF,dm=dm):
    teta,phi=angles
    coords=euler2coords(teta,phi,1.695741)
    return Vtot(coords,mol=HF,dm=dm)

In [12]:
prot_angles=minimize(Vtot_fl,(1.4,0),args=(1.695741)).x

In [13]:
prot_angles

array([ 2.08705644e+00, -1.33294520e-07])

In [14]:
prot_site=euler2coords(*prot_angles,1.695741)
prot_site

array([ 1.47473742e+00, -1.96574417e-07, -8.37070537e-01])

In [15]:
prot_site=minimize(Vtot,[0,.5,-1.3]).x
norm(prot_site),anglecenter(np.vstack([HF.atom_coords(),prot_site])) #*180/np.pi

(1.5510757001436768, 118.422)

# How to develop the basis functions or do none.


In [16]:
pHF=FcM(fcs=[0,0,-1],atom="F 0 0 0;H 0 0 1.695741; H {} {} {}".format(prot_site[0],prot_site[1],prot_site[2]),unit="bohr",
          basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])},charge=1)
pHF.charge=0

In [17]:
pHFmf=scf.RHF(pHF)
pHFmf.scf(dm0=pHFmf.init_guess_by_1e())

converged SCF energy = -100.065874806345


-100.06587480634477

In [18]:
afHF=aaff(pHFmf,[-1,0,1])+alc_deriv_grad_nuc(pHF,[-1,0,1])
apHF=AP(pHFmf,sites=[0,2])
apHF.build_all()
apHF.APDFT3([-1,1])

-75.96255971334608

In [19]:
pHFm=FcM(fcs=[.1,0,-1.1],atom="F 0 0 0;H 0 0 1.695741; H {} {} {}".format(prot_site[0],prot_site[1],prot_site[2]),unit="bohr",
          basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])},charge=1)
pHFmfm=scf.RHF(pHFm)
pHFmfm.scf(dm0=pHFmfm.init_guess_by_1e())

converged SCF energy = -102.756154138897


-102.7561541388971

In [20]:
afHFm=aaff(pHFmfm,[-1,0,1])+alc_deriv_grad_nuc(pHFm,[-1,0,1])
apHFm=AP(pHFmfm,sites=[0,2])
apHFm.build_all()
apHFm.APDFT3([-1,1])

-78.1262277056546

In [21]:
pHFp=FcM(fcs=[-.1,0,-.9],atom="F 0 0 0;H 0 0 1.695741; H {} {} {}".format(prot_site[0],prot_site[1],prot_site[2]),unit="bohr",
          basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])},charge=1)
pHFmfp=scf.RHF(pHFp)
pHFmfp.scf(dm0=pHFmfp.init_guess_by_1e())

converged SCF energy = -97.4269980240937


-97.4269980240937

In [22]:
afHFp=aaff(pHFmfp,[-1,0,1])+alc_deriv_grad_nuc(pHFp,[-1,0,1])
apHFp=AP(pHFmfp,sites=[0,2])
apHFp.build_all()

In [23]:
HFh0=pHFmf.Hessian().hess()
HFhm=pHFmfm.Hessian().hess()
HFhp=pHFmfp.Hessian().hess()

In [24]:
HFht=HFh0+(HFhp-HFhm)/.2+(HFhp+HFhm-2*HFh0)/2/.01

In [25]:
HFgt=afHF+(afHFp-afHFm)/.2/2+(afHFp+afHFm-2*afHF)/6/.01

In [26]:
HFet=apHF.APDFT3([-1,1])+absec("F","O")+(apHFp.third_deriv([-1,1])-apHFm.third_deriv([-1,1]))/.2/24+\
       (apHFp.third_deriv([-1,1])+apHFm.third_deriv([-1,1])-2*apHF.third_deriv([-1,1]))/.01/120                                  

In [27]:
mpa_pb_with_energy(pHF.atom_coords(),['O','H','H'],HFgt,HFht,HFet)


[-0.04837561 -0.17835542  0.02442416]
[ 0.00815086  0.0489941  -0.00733526] [-0.00733526]
[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)]


(array([1.695741  , 1.5510757 , 2.06684846]),
 array([ 0.06841579,  0.12829248, -0.18505165]),
 array([1.76415679, 1.67936819, 1.88179681]),
 array([1.77230765, 1.72836228, 1.87446155]),
 -76.00198556036152)

In [28]:
1.8744619*180/np.pi,apHF.APDFT3([-1,1]),HFet

(107.39875572807337, -75.96255971334608, -75.98380436718725)

# H$_2$O

In [29]:
oh2lst=[]
for x in range(3):
    oh2lst.append("OHH"[x])
    oh2lst.extend(oh2optg[x].tolist())  
oh2srt=(("{} "*4+";")*3)[:-1].format(*oh2lst)

In [30]:
OH2=gto.M(atom=oh2srt,unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mfoh2 = scf.RHF(OH2)
mfoh2.scf()
dmoh2=mfoh2.make_rdm1()

converged SCF energy = -76.0633193328125


In [31]:
ps_h2o=minimize(Vtot,np.array([1.5,0,0]),args=(OH2,dmoh2)).x

In [32]:
pH2O=FcM(fcs=[0,0,0,-1],atom=oh2srt+"; H {} {} {}".format(*ps_h2o),\
         unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])},charge=1)
pH2Omf=scf.RHF(pH2O)
pH2Omf.scf(pH2Omf.init_guess_by_1e())

converged SCF energy = -76.0638566271091


-76.06385662710909

In [33]:
pH2Op=FcM(fcs=[-.1,0,0,-.9],atom=oh2srt+"; H {} {} {}".format(*ps_h2o),\
         unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])},charge=1)
pH2Omfp=scf.RHF(pH2Op)
pH2Omfp.scf(pH2Omfp.init_guess_by_1e())

converged SCF energy = -73.8541068480512


-73.85410684805119

In [34]:
pH2Om=FcM(fcs=[.1,0,0,-1.1],atom=oh2srt+"; H {} {} {}".format(*ps_h2o),\
         unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])},charge=1)
pH2Omfm=scf.RHF(pH2Om)
pH2Omfm.scf(pH2Omfm.init_guess_by_1e())

converged SCF energy = -78.3219845318812


-78.32198453188123

In [35]:
afph2o=aaff(pH2Omf,[-1,0,0,1])+alc_deriv_grad_nuc(pH2O,[-1,0,0,1])
afph2op=aaff(pH2Omfp,[-1,0,0,1])+alc_deriv_grad_nuc(pH2Op,[-1,0,0,1])
afph2om=aaff(pH2Omfm,[-1,0,0,1])+alc_deriv_grad_nuc(pH2Om,[-1,0,0,1])

In [36]:
afph2ot=afph2o+(afph2op-afph2om)/.2/2+(afph2op+afph2om-2*afph2o)/6/.01

In [37]:
pH2Oh=pH2Omf.Hessian().hess()
pH2Ohp=pH2Omfp.Hessian().hess()
pH2Ohm=pH2Omfm.Hessian().hess()

In [38]:
pH2Oht=pH2Oh+(pH2Ohp-pH2Ohm)/.2+(pH2Ohp+pH2Ohm-2*pH2Oh)/2/.01

In [39]:
apH2O=AP(pH2Omf,sites=[0,3])
apH2O.build_all()
apH2Op=AP(pH2Omfp,sites=[0,3])
apH2Op.build_all()
apH2Om=AP(pH2Omfm,sites=[0,3])
apH2Om.build_all()

In [40]:
pH2Oet=apH2O.APDFT3([-1,1])+absec("O","N")+(apH2Op.third_deriv([-1,1])-apH2Om.third_deriv([-1,1]))/.2/24+\
       (apH2Op.third_deriv([-1,1])+apH2Om.third_deriv([-1,1])-2*apH2O.third_deriv([-1,1]))/.01/120                   

In [41]:
mpa_pb_with_energy(pH2O.atom_coords(),['N','H','H','H'],afph2ot,pH2Oht,pH2Oet)

[-0.0607401  -0.06073964 -0.18325647 -0.01672616  0.01325029  0.01325343]
[ 0.01569569  0.01569553  0.06125341 -0.00859096 -0.01680766 -0.01680616] [-0.00859055 -0.0168064  -0.01680769]
[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(2, 0, 3, weak=0)]


(array([1.77561   , 1.77561   , 1.61807455, 1.85690619, 2.07368803,
        2.07370923]),
 array([ 0.08872801,  0.08872053,  0.15143116,  0.06121351, -0.15000984,
        -0.15003251]),
 array([1.86433801, 1.86433053, 1.76950571, 1.9181197 , 1.92367819,
        1.92367672]),
 array([1.88003371, 1.88002606, 1.83075913, 1.90952874, 1.90687052,
        1.90687056]),
 -56.16235471988964)

In [82]:
mpa_pb_with_energy(pH2O.atom_coords(),['N','H','H','H'],afph2ot,pH2Oht,pH2Oet)

[-0.0607401  -0.06073964 -0.18325647 -0.01672616  0.01325029  0.01325343]
[ 0.01569569  0.01569553  0.06125341 -0.00859096 -0.01680766 -0.01680616] [-0.00859055 -0.0168064  -0.01680769]
[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(2, 0, 3, weak=0)]


(array([1.77561   , 1.77561   , 1.61807455, 1.85690619, 2.07368803,
        2.07370923]),
 array([ 0.08872792,  0.08873024,  0.15143087,  0.06117837, -0.15001641,
        -0.15003032]),
 array([1.86433792, 1.86434024, 1.76950542, 1.91808456, 1.92367162,
        1.92367892]),
 array([1.88003362, 1.88003577, 1.83075883, 1.90949359, 1.90686396,
        1.90687275]),
 -56.1623547968167)

In [43]:
1.90951409*180/np.pi,1.90686496*180/np.pi

(109.40709827776404, 109.25531430938256)

#     NH$_3$

In [44]:
ch4optg,nh3optg,oh2optg,fhoptg=np.load("optgeoms.npy")
nh3lst=[]
for x in range(4):
    nh3lst.append("NHHH"[x])
    nh3lst.extend(nh3optg[x].tolist())
nh3srt=(("{} "*4+";")*4)[:-1].format(*nh3lst)
NH3=gto.M(atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf3 = scf.RHF(NH3)
mf3.scf()

converged SCF energy = -56.2221836146943


-56.22218361469434

In [45]:
nh3dm=mf3.make_rdm1()
ps_nh3=minimize(Vtot,np.array([0,-1.5,.3]),args=(NH3,nh3dm)).x

In [46]:
ps_nh3

array([ 6.80657852e-07, -1.69932458e+00, -2.79748075e-01])

In [47]:
pNH3=FcM(fcs=[0,0,0,0,-1],atom=nh3srt+"; H {} {} {} ".format(*ps_nh3),unit="bohr",\
         basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)
pNH3p=FcM(fcs=[-.1,0,0,0,-.9],atom=nh3srt+"; H {} {} {} ".format(*ps_nh3),unit="bohr",\
         basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)
pNH3m=FcM(fcs=[.1,0,0,0,-1.1],atom=nh3srt+"; H {} {} {} ".format(*ps_nh3),unit="bohr",\
         basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)
pNH3mf=scf.RHF(pNH3)
pNH3mf.scf(dm0=pNH3mf.init_guess_by_1e())
pNH3mfm=scf.RHF(pNH3m)
pNH3mfm.scf(dm0=pNH3mfm.init_guess_by_1e())
pNH3mfp=scf.RHF(pNH3p)
pNH3mfp.scf(dm0=pNH3mfp.init_guess_by_1e())

converged SCF energy = -56.2226234868909
converged SCF energy = -58.0803722521554
converged SCF energy = -54.409898250772


-54.40989825077203

In [48]:
NH3h=pNH3mf.Hessian().hess()
NH3hp=pNH3mfp.Hessian().hess()
NH3hm=pNH3mfm.Hessian().hess()

In [49]:
NH3ht=NH3h+(NH3hp-NH3hm)/.2+(NH3hp+NH3hm-2*NH3h)/.01/2

In [52]:
apNH3=AP(pNH3mf,sites=[0,4])
apNH3.build_all()
apNH3p=AP(pNH3mfp,sites=[0,4])
apNH3p.build_all()
apNH3m=AP(pNH3mfm,sites=[0,4])
apNH3m.build_all()

In [53]:
NH3et=apNH3.APDFT3([-1,1])+absec("N","C")+(apNH3p.third_deriv([-1,1])-apNH3m.third_deriv([-1,1]))/.2/24+\
    (apNH3p.third_deriv([-1,1])+apNH3m.third_deriv([-1,1])-2*apNH3.third_deriv([-1,1]))/120/.01

In [63]:
afpnh3=aaff(pNH3mf,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3,[-1,0,0,0,1])
afpnh3p=aaff(pNH3mfp,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3p,[-1,0,0,0,1])
afpnh3m=aaff(pNH3mfm,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3m,[-1,0,0,0,1])

In [64]:
NH3gt=afpnh3+(afpnh3p-afpnh3m)/.2/2+(afpnh3p+afpnh3m-2*afpnh3)/.01/6

In [76]:
mpa_pb_with_energy(pNH3.atom_coords(),['C','H','H','H','H'],NH3gt,NH3ht,NH3et)

[-0.07550262 -0.07550305 -0.07550303 -0.17569327 -0.00169501 -0.00169501
  0.00162279 -0.00169611  0.00162227  0.00162239]
[ 0.03394816  0.03394844  0.03394813  0.0796015   0.003842    0.00383304
 -0.0036746   0.00383521 -0.00367245 -0.0036683 ] [0.60161229 0.60160324 0.6209231  0.60163641 0.62098485 0.62098885]
[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Bond(0, 4, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(1, 0, 4, weak=0), Angle(2, 0, 3, weak=0), Angle(2, 0, 4, weak=0), Angle(3, 0, 4, weak=0)]


(array([1.88552   , 1.88552   , 1.88552   , 1.72219714, 1.88833539,
        1.88833539, 1.93239388, 1.88846925, 1.93239252, 1.93239321]),
 array([ 0.12831285,  0.12831338,  0.12831156,  0.17569451,  0.01661676,
         0.01662388, -0.01591484,  0.01664629, -0.0159162 , -0.01591095]),
 array([2.01383285, 2.01383338, 2.01383156, 1.89789165, 1.90495214,
        1.90495926, 1.91647904, 1.90511554, 1.91647632, 1.91648226]),
 array([2.04778101, 2.04778182, 2.04777969, 1.97749315, 1.90879415,
        1.9087923 , 1.91280443, 1.90895074, 1.91280387, 1.91281396]),
 -40.16255242436402)

In [79]:
1.90879415*180/np.pi,1.91280443*180/np.pi

(109.36584875426138, 109.5956208729271)

In [80]:
afpnh3=aaff(pNH3mf,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3,[-1,0,0,0,1])
afpnh3p=aaff(pNH3mfp,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3p,[-1,0,0,0,1])
afpnh3m=aaff(pNH3mfm,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3m,[-1,0,0,0,1])
NH3gt=afpnh3+(afpnh3p-afpnh3m)/.2/2+(afpnh3p+afpnh3m-2*afpnh3)/.01/6
mpa_pb_with_energy(pNH3.atom_coords(),['C','H','H','H','H'],NH3gt,NH3ht,NH3et)

[-0.07550262 -0.07550305 -0.07550303 -0.17569327 -0.00169501 -0.00169501
  0.00162279 -0.00169611  0.00162227  0.00162239]
[ 0.03394816  0.03394844  0.03394813  0.0796015   0.003842    0.00383304
 -0.0036746   0.00383521 -0.00367245 -0.0036683 ] [-0.6416898  -0.64169949 -0.67817682 -0.64173083 -0.67824031 -0.67823664]
[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Bond(0, 4, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(1, 0, 4, weak=0), Angle(2, 0, 3, weak=0), Angle(2, 0, 4, weak=0), Angle(3, 0, 4, weak=0)]


(array([1.88552   , 1.88552   , 1.88552   , 1.72219714, 1.88833539,
        1.88833539, 1.93239388, 1.88846925, 1.93239252, 1.93239321]),
 array([ 0.12831285,  0.12831338,  0.12831156,  0.17569451,  0.01661676,
         0.01662388, -0.01591484,  0.01664629, -0.0159162 , -0.01591095]),
 array([2.01383285, 2.01383338, 2.01383156, 1.89789165, 1.90495214,
        1.90495926, 1.91647904, 1.90511554, 1.91647632, 1.91648226]),
 array([2.04778101, 2.04778182, 2.04777969, 1.97749315, 1.90879415,
        1.9087923 , 1.91280443, 1.90895074, 1.91280387, 1.91281396]),
 -40.16255242436399)

In [81]:
afpnh3=aaff(pNH3mf,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3,[-1,0,0,0,1])
afpnh3p=aaff(pNH3mfp,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3p,[-1,0,0,0,1])
afpnh3m=aaff(pNH3mfm,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3m,[-1,0,0,0,1])
NH3gt=afpnh3+(afpnh3p-afpnh3m)/.2/2+(afpnh3p+afpnh3m-2*afpnh3)/.01/6
mpa_pb_with_energy(pNH3.atom_coords(),['C','H','H','H','H'],NH3gt,NH3ht,NH3et)

[-0.07550262 -0.07550305 -0.07550303 -0.17569327 -0.00169501 -0.00169501
  0.00162279 -0.00169611  0.00162227  0.00162239]
[ 0.03394816  0.03394844  0.03394813  0.0796015   0.003842    0.00383304
 -0.0036746   0.00383521 -0.00367245 -0.0036683 ] [-0.71119377 -0.7112035  -0.75080004 -0.71123844 -0.75087053 -0.75086689]
[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Bond(0, 4, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(1, 0, 4, weak=0), Angle(2, 0, 3, weak=0), Angle(2, 0, 4, weak=0), Angle(3, 0, 4, weak=0)]


(array([1.88552   , 1.88552   , 1.88552   , 1.72219714, 1.88833539,
        1.88833539, 1.93239388, 1.88846925, 1.93239252, 1.93239321]),
 array([ 0.12831285,  0.12831338,  0.12831156,  0.17569451,  0.01661676,
         0.01662388, -0.01591484,  0.01664629, -0.0159162 , -0.01591095]),
 array([2.01383285, 2.01383338, 2.01383156, 1.89789165, 1.90495214,
        1.90495926, 1.91647904, 1.90511554, 1.91647632, 1.91648226]),
 array([2.04778101, 2.04778182, 2.04777969, 1.97749315, 1.90879415,
        1.9087923 , 1.91280443, 1.90895074, 1.91280387, 1.91281396]),
 -40.16255242436401)

In [73]:
afpnh3=aaff(pNH3mf,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3,[-1,0,0,0,1])
afpnh3p=aaff(pNH3mfp,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3p,[-1,0,0,0,1])
afpnh3m=aaff(pNH3mfm,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3m,[-1,0,0,0,1])
NH3gt=afpnh3+(afpnh3p-afpnh3m)/.2/2+(afpnh3p+afpnh3m-2*afpnh3)/.01/6
mpa_pb_with_energy(pNH3.atom_coords(),['C','H','H','H','H'],NH3gt,NH3ht,NH3et)

[-0.07550262 -0.07550305 -0.07550303 -0.17569327 -0.00169501 -0.00169501
  0.00162279 -0.00169611  0.00162227  0.00162239]
[ 0.03394816  0.03394844  0.03394813  0.0796015   0.003842    0.00383304
 -0.0036746   0.00383521 -0.00367245 -0.0036683 ] [0.60161229 0.60160324 0.6209231  0.60163641 0.62098485 0.62098885]
[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Bond(0, 4, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(1, 0, 4, weak=0), Angle(2, 0, 3, weak=0), Angle(2, 0, 4, weak=0), Angle(3, 0, 4, weak=0)]


(array([1.88552   , 1.88552   , 1.88552   , 1.72219714, 1.88833539,
        1.88833539, 1.93239388, 1.88846925, 1.93239252, 1.93239321]),
 array([0.12830657, 0.1283093 , 0.12831708, 0.17569631, 1.13749466,
        1.1374734 , 1.15523182, 1.13755178, 1.15534986, 1.15535355]),
 array([2.01382657, 2.0138293 , 2.01383708, 1.89789345, 3.02583005,
        3.02580878, 3.0876257 , 3.02602103, 3.08774238, 3.08774675]),
 array([2.04777473, 2.04777774, 2.04778521, 1.97749495, 3.02967205,
        3.02964182, 3.0839511 , 3.02985623, 3.08406993, 3.08407846]),
 -40.1625524711306)